<h1 style="color: #00BFFF;">Python-SQL Connection</h1>

In [1]:
# 📚 Basic libraries
import pandas as pd # data manipulation
import numpy as np # numerical operations
import warnings # warning messages management

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

# 🌐 New Libraries
from getpass import getpass # get password without showing it
import pymysql # MySQL connection

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

In [2]:
def create_connection():
    password = getpass("Please, kindly insert your password: ")
    cnx = pymysql.connect(user='root', password=password, host='localhost')
    
    if cnx.open:
        print("Connection successfully opened.")
        print()
        cursor = cnx.cursor()
        cursor.execute("SHOW DATABASES")
        databases = cursor.fetchall()
        print("Available databases:")
        db_list = [db[0] for db in databases]
        for db in db_list:
            print(f"- {db}")
        print()
    else:
        print("Failed to open connection.")
        print()
        return None
    
    while True:
        database = input("Please, kindly insert your database name: ")
        if database in db_list:
            cnx.select_db(database)
            break
        else:
            print("Invalid database name. Please choose from the available databases.")
            print()
    
    return cnx

<h2 style="color: #00BFFF;">[1] Connecting to MySQL Workbench</h2>

In [3]:
# To test it, use a DB from Drive: LibraryDB, AppleDB or w3schools
connection = create_connection()

Connection successfully opened.

Available databases:
- amz_uk_price_prediction_dataset
- AppleDB
- information_schema
- iss_db_final
- LibraryDB
- mysql
- performance_schema
- sakila
- sys
- w3schools



<h2 style="color: #00BFFF;">[2] ETL: Data Storage</h2>

In [ ]:
df = pd.read_csv("clean_data.csv")

In [13]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [17]:
cursor = connection.cursor()

In [18]:
cursor.execute("create database if not exists diplomacyDB;")
connection.commit()

In [ ]:
connection.select_db("diplomacyDB")
connection.commit()

In [19]:
df.head(2)

,year,cname_send,main_posting,title,gender,cname_receive,ccode_send,ccodealp_send,ccodeCOW_send,region_send,GME_send,v2lgfemleg_send,FFP_send,ccode_receive,ccodealp_receive,ccodeCOW_receive,region_receive,GME_receive,FFP_receive
0,1968,Afghanistan,1,3,0,China,4,AFG,700,1,1,2.0,0,156,CHN,710,1,0,0
1,1968,Afghanistan,1,3,0,Czechoslovakia,4,AFG,700,1,1,2.0,0,200,CSK,315,3,0,0


## Create your tables:

In [20]:
df.columns

Index(['year', 'cname_send', 'main_posting', 'title', 'gender',
       'cname_receive', 'ccode_send', 'ccodealp_send', 'ccodeCOW_send',
       'region_send', 'GME_send', 'v2lgfemleg_send', 'FFP_send',
       'ccode_receive', 'ccodealp_receive', 'ccodeCOW_receive',
       'region_receive', 'GME_receive', 'FFP_receive'],
      dtype='object')

In [21]:
df.isna().sum()

year                0
cname_send          0
main_posting        0
title               0
gender              0
cname_receive       0
ccode_send          0
ccodealp_send       0
ccodeCOW_send       0
region_send         0
GME_send            0
v2lgfemleg_send     0
FFP_send            0
ccode_receive       0
ccodealp_receive    0
ccodeCOW_receive    0
region_receive      0
GME_receive         0
FFP_receive         0
dtype: int64

In [24]:
df.head(2)

,year,cname_send,main_posting,title,gender,cname_receive,ccode_send,ccodealp_send,ccodeCOW_send,region_send,GME_send,v2lgfemleg_send,FFP_send,ccode_receive,ccodealp_receive,ccodeCOW_receive,region_receive,GME_receive,FFP_receive
0,1968,Afghanistan,1,3,0,China,4,AFG,700,1,1,2.0,0,156,CHN,710,1,0,0
1,1968,Afghanistan,1,3,0,Czechoslovakia,4,AFG,700,1,1,2.0,0,200,CSK,315,3,0,0


In [25]:
create_table_query = """
CREATE TABLE IF NOT EXISTS gender_diplomacy (
    year INT,
    cname_send VARCHAR(255),
    main_posting INT,
    title INT,
    gender INT,
    cname_receive VARCHAR(255),
    ccode_send INT,
    ccodealp_send VARCHAR(255),
    ccodeCOW_send INT,
    region_send INT,
    GME_send INT,
    v2lgfemleg_send FLOAT,
    FFP_send INT,
    ccode_receive INT,
    ccodealp_receive VARCHAR(255),
    ccodeCOW_receive INT,
    region_receive INT,
    GME_receive INT,
    FFP_receive INT
);
"""

In [27]:
connection.select_db("diplomacyDB")
connection.commit()

In [28]:
cursor = connection.cursor()

In [29]:
cursor.execute(create_table_query)
connection.commit()

<h3 style="color: #00BFFF;">Insert the values</h3>

In [30]:
columns = ", ".join(df.columns)
placeholders = ", ".join(["%s"] * len(df.columns))

In [33]:
insert_query = f"""
INSERT INTO gender_diplomacy ({columns})
VALUES ({placeholders})
"""

# Change the table name

In [35]:
for _, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))
connection.commit()

In [42]:
df.isna().sum()

artist           0
birth            0
death            0
nationality      0
gender           0
painting_date    0
height           0
width            0
dtype: int64

In [43]:
df.shape

(77376, 8)